In [1]:
import sys
sys.path.append("/Users/wj/新浪数科/模型开发/")

import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
import scorecardpy as sc
import pickle
from model_tools.metrics import roc_auc_score, ks
from model_tools.ScoreCard import model_helper
from model_tools.Model import model_utils

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_data = pd.read_csv("../input/FT4_0_MODEL_DATA_230418.csv")

In [3]:
model_data["diff_pass_credit_first_loan_time"].head()

0    12894731.0
1    73240190.0
2         229.0
3     1472304.0
4         438.0
Name: diff_pass_credit_first_loan_time, dtype: float64

In [4]:
loan_behavior_data = pd.read_csv("../input/ft4_0_loan_behavior_variables_20230509.csv")
loan_behavior_variables = list(loan_behavior_data.columns[8:])
model_data.drop(['dpd30_at_mob6'], axis=1, inplace=True)
model_data = model_data.merge(loan_behavior_data[['biz_id', 'dpd15_at_mob1', 'dpd30_at_mob6']+loan_behavior_variables], left_on='loan_id', right_on='biz_id', how='left')

In [5]:
model_data.to_csv("../output/FT3_2_MODEL_DATA_230510.csv", index=False)

In [6]:
loan_behavior_variables = ['loan_apply_count',
 'loan_apply_count_15d',
 'loan_apply_count_30d',
 'loan_apply_count_60d',
 'loan_apply_count_90d',
 'loan_apply_count_180d',
 'loan_apply_count_360d',
 'loan_term3_apply_count',
 'loan_term6_apply_count',
 'loan_term12_apply_count',
 'loan_hour06_apply_count',
 'loan_hour12_apply_count',
 'loan_hour18_apply_count',
 'loan_hour24_apply_count',
 'first_reject_to_now_days',
 'recent_reject_to_now_days']

In [7]:
# 额度使用率
limit_used_data = pd.read_csv("../input/limit_used_rate_data.csv")

In [8]:
limit_used_data['limit_used_rate'] = limit_used_data['apply_amount']/limit_used_data['orders_curr_total_limit']

In [9]:
loan_behavior_variables_2 = [
    'bankcardauthcount', 'communicationauthcount',
   'emercontactcount', 'bank_phone_same', 'location_same_province',
   'diff_storage_mobile_time', 'diff_storage_credit_time',
   'diff_mobile_bank_time', 'diff_bank_carrier_time',
   'diff_carrier_credit_time', 'first_credit_hour', 'first_credit_week',
   'first_credit_weekend', 
   'first_credit_succ_hour', 'first_credit_succ_week',
   'first_credit_succ_weekend', 'first_loan_hour', 'first_loan_week',
   'first_loan_weekend', 'diff_wb_reg_first_loan_time',
   'diff_wb_reg_first_credit_time', 'city_level_card_num',
   'city_level_mobile_num', 'city_level_bank_num', 'mobile_unbound_times',
    'user_name_is_figure_or_alphabet',
   'emer_contact_user1_name_is_figure_or_alphabet',
   'emer_contact_user2_name_is_figure_or_alphabet',
   'user_emer1_emer2_name_no_equal', 'user_emer1_emer2_mobile_no_equal',
   'id_card_expire_diff_now_days', 'cardfrontkeysize', 'cardbackkeysize',
   'contracts_count', 
   'limit_used_rate', 
   'wheneverloan_loan_info_diff_pass_credit_first_loan_time',
]

In [10]:
limit_used_data.query("now_host_amount_rate==now_host_amount_rate")[['now_host_amount_rate', 'limit_used_rate', 'orders_curr_total_limit', 'apply_amount']]

,now_host_amount_rate,limit_used_rate,orders_curr_total_limit,apply_amount
0,0.0200,0.020000,50000.0,1000.0
3,1.0000,1.000000,2200.0,2200.0
13,0.2020,0.202020,49500.0,10000.0
16,0.0279,0.027855,35900.0,1000.0
20,1.0000,1.000000,2000.0,2000.0
...,...,...,...,...
101549,0.0167,0.016667,30000.0,500.0
101552,0.3333,0.333333,15000.0,5000.0
101570,0.7692,0.769231,13000.0,10000.0
101586,1.0000,NaN,NaN,NaN


In [11]:
limit_used_data.rename(columns={'biz_id': 'loan_id'}, inplace=True)
model_data = model_data.merge(limit_used_data[['loan_id']+loan_behavior_variables_2], on='loan_id', how='left')

In [12]:
def f_mi(x, target):
    d = []
    d.append(x['cnt'].sum())
    d.append(x[target].sum())
    d.append(x[target].mean())
    return pd.Series(d, index=['count', 'bad', 'bad_rate'])

model_data['cnt'] = 1
model_data.query("dpd30_at_mob3!=-1").groupby(["apply_month"]).apply(f_mi, "dpd30_at_mob3")

,count,bad,bad_rate
apply_month,,,
2022-04,11077.0,542.0,0.048930
2022-05,15260.0,776.0,0.050852
2022-06,13108.0,642.0,0.048978
2022-07,12263.0,487.0,0.039713
2022-08,11168.0,532.0,0.047636
2022-09,10662.0,510.0,0.047833
2022-10,10803.0,598.0,0.055355


In [13]:
model_data.query("dpd30_at_mob6!=[-1, -9999]").groupby(["apply_month"]).apply(f_mi, "dpd30_at_mob6")

,count,bad,bad_rate
apply_month,,,
2022-04,10179.0,1220.0,0.119855
2022-05,13905.0,1705.0,0.122618
2022-06,12664.0,1600.0,0.126342
2022-07,11375.0,1271.0,0.111736
2022-08,10372.0,1107.0,0.106730
2022-09,9616.0,760.0,0.079035
2022-10,2777.0,242.0,0.087144


In [14]:
model_data.query("dpd15_at_mob1!=[-1, -9999]").groupby(["apply_month"]).apply(f_mi, "dpd15_at_mob1")

,count,bad,bad_rate
apply_month,,,
2022-04,12665.0,123.0,0.009712
2022-05,17443.0,209.0,0.011982
2022-06,15462.0,151.0,0.009766
2022-07,13896.0,103.0,0.007412
2022-08,12872.0,108.0,0.008390
2022-09,11795.0,113.0,0.009580
2022-10,12056.0,130.0,0.010783


In [18]:
nb_variables = ["gender", "age", "idcardpre6", "diff_pass_credit_first_loan_time"]
wb_score = ['wb_v2_0_confidence', 'wb_v2_0_prob']
br_variables = [x for x in model_data.columns if x.startswith("als_") and not x.endswith("_status")]
wb_variables = [x for x in model_data.columns if x.startswith("sina") and not x.endswith("_status")] + ['now_duration_weibo_reg']
ronghui_p_variables = [x for x in model_data.columns if x.startswith("ronghui_p") and not x.endswith("_status")]
tx_risk_score = ['tx_riskscore']
blz_tz_score = ['bileizhen_tuzhi_10_score']
youmeng_score = ["youmeng_bank_score"]
jd_variables = [x for x in model_data.columns if x.startswith("jd_") and not x.endswith("_status")]
hudao_variables = [x for x in model_data.columns if x.startswith("hd_") and not x.endswith("_status")]
dxm_score = ['dxm_general_fraudc_v3_score']
bw_score = ['bw_b_score_v3']
ks_variables = [x for x in model_data.columns if x.startswith("ppde_") and not x.endswith("_status")]
jd_variables1 = [
    'jd_ppre_has_child_score',
    'jd_ppre_consm_tob_level',
    'jd_ppre_purch_score',
    'jd_ppre_lux_pref_score',
    'jd_ppre_have_house_score'
]
jd_variables2 = [
    'jd_ppre_intst_rate_sens_level',
    'jd_ppre_credit_asst_level',
    'jd_ppre_debit_level',
    'jd_ppre_device_price_level',
]

loan_behavior_variables = ['loan_apply_count',
 'loan_apply_count_15d',
 'loan_apply_count_30d',
 'loan_apply_count_60d',
 'loan_apply_count_90d',
 'loan_apply_count_180d',
 'loan_apply_count_360d',
 'loan_term3_apply_count',
 'loan_term6_apply_count',
 'loan_term12_apply_count',
 'loan_hour06_apply_count',
 'loan_hour12_apply_count',
 'loan_hour18_apply_count',
 'loan_hour24_apply_count',
 'first_reject_to_now_days',
 'recent_reject_to_now_days']

condicate_variables = nb_variables +\
    wb_score +\
    br_variables +\
    ronghui_p_variables +\
    tx_risk_score +\
    youmeng_score +\
    hudao_variables +\
    dxm_score +\
    bw_score +\
    ks_variables +\
    jd_variables2 +\
    loan_behavior_variables +\
    loan_behavior_variables_2

In [19]:
# data_report = model_helper.data_report(model_data[condicate_variables])

In [20]:
# from woe_bin import woe_bin

# temp = model_data.query("dpd30_at_mob6 != [-9999, -1]")[condicate_variables+["dpd30_at_mob6"]]
# mono_woe = woe_bin(indata=temp, target="dpd30_at_mob6", min_group_rate=0.05, max_bin=6, bin_method='mono', alg_method='iv')
# mapiv_temp = mono_woe.split_data()

In [18]:
mapiv_temp.sort_values(['iv', 'varname'], ascending=[False, False], inplace=True)

In [19]:
# mapiv_temp.to_csv("../output/mono_woe_mob6_230509.csv", index=False)
mapiv_temp = pd.read_csv("../output/mono_woe_mob6_230509.csv")

In [20]:
select_iv_variables = [x for x in mapiv_temp.query("iv >= 0.001")['varname'].drop_duplicates().tolist() if x in condicate_variables]

In [21]:
result = """"""
result += "def process(model_data):\n"
result += "  import numpy as np\n"
for col in select_iv_variables:
    bins_info_temp = mapiv_temp.query(f"varname=='{col}'")
    have_null = 1-int(bins_info_temp['bin'].tolist()[0])
    # woe transformer
    for index, left, right, woe in zip(bins_info_temp['bin'].tolist(), 
                                bins_info_temp['ll'].tolist(),
                                bins_info_temp['ul'].tolist(),
                                bins_info_temp['woe'].tolist()):
        
        if have_null:
            if str(left) == "nan" and str(right) == "nan":
                result += f"  model_data['W_{col}'] = \\\n    np.where(model_data['{col}'].isnull(),     {woe},\n"
            elif str(right) == "inf":
                result += f"  {woe}" + ")"*int(index) + "\n\n"
            else:
                result += f"    np.where(model_data['{col}'] < {right},        {woe},\n"
        else:
            if int(index)==1:
                result += f"  model_data['W_{col}'] = \\\n    np.where(model_data['{col}'] <{right},     {woe},\n"
            elif str(right) == "inf":
                result += f"  {woe}" + ")"*(int(index)-1) + "\n\n"
            else:
                result += f"    np.where(model_data['{col}'] < {right},        {woe},\n"
            
result += "  return model_data"
 
print(result, file=open("condition_calculate_mob6_230509.py", "w+"))

In [22]:
import condition_calculate_mob6_230509
from imp import reload
reload(condition_calculate_mob6_230509)

model_data = condition_calculate_mob6_230509.process(model_data)

In [23]:
dev_sample1 = model_data.query("apply_month>='2022-04' and apply_month<='2022-07' and dpd30_at_mob6!=[-1, -9999]").reset_index(drop=True)
oot_sample1 = model_data.query("apply_month>='2022-08' and apply_month<='2022-10' and dpd30_at_mob6!=[-1, -9999]").reset_index(drop=True)

In [24]:
from sklearn.model_selection import train_test_split, StratifiedKFold

target = "dpd30_at_mob6"
train, validation = train_test_split(dev_sample1, test_size=0.3, random_state=42)
print("Train SAMPLE SIZE: %d, BAD: %d, BAD RATIO: %.4f" %
      (train.shape[0], train[target].sum(), train[target].mean()))
print("Validation SAMPLE SIZE: %d, BAD: %d, BAD RATIO: %.4f" %
      (validation.shape[0], validation[target].sum(), validation[target].mean()))

Train SAMPLE SIZE: 33686, BAD: 4057, BAD RATIO: 0.1204
Validation SAMPLE SIZE: 14437, BAD: 1739, BAD RATIO: 0.1205


In [25]:
condicate_variables1 = nb_variables +\
    wb_score +\
    br_variables +\
    tx_risk_score +\
    youmeng_score +\
    dxm_score +\
    bw_score +\
    ks_variables +\
    jd_variables2 +\
    loan_behavior_variables +\
    loan_behavior_variables_2

condicate_variables2 = nb_variables +\
    wb_score +\
    br_variables +\
    ronghui_p_variables +\
    tx_risk_score +\
    youmeng_score +\
    dxm_score +\
    bw_score +\
    ks_variables +\
    jd_variables2 +\
    loan_behavior_variables +\
    ['limit_used_rate']

input_cols = [x for x in mapiv_temp.query("varname==@condicate_variables2 and iv >= 0.01")['varname'].drop_duplicates().tolist()]
input_woe_variables = ['W_'+x for x in input_cols]

In [26]:
xgb_model = XGBClassifier(
    base_score=0.5,
    colsample_bylevel=1,
    colsample_bytree=0.9,
    gamma=0.1,
    learning_rate=0.08,
    max_depth=4,
    min_child_weight=150,
    missing=None,
    n_estimators=120,
    nthread=-1,
    n_jobs=-1,
    objective='binary:logistic',
    reg_alpha=0.1,
    reg_lambda=0.1,
    scale_pos_weight=1,
    seed=42,
    silent=True,
    subsample=0.9
)


xgb_model.fit(train[input_woe_variables], train[target], eval_set=[(train[input_woe_variables], train[target]), (validation[input_woe_variables], validation[target])],
         eval_metric='auc', early_stopping_rounds=20, verbose=20)

[0]	validation_0-auc:0.696956	validation_1-auc:0.689117
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[20]	validation_0-auc:0.745373	validation_1-auc:0.735571
[40]	validation_0-auc:0.761193	validation_1-auc:0.747949
[60]	validation_0-auc:0.771151	validation_1-auc:0.754463
[80]	validation_0-auc:0.779315	validation_1-auc:0.760282
[100]	validation_0-auc:0.785971	validation_1-auc:0.763725
[119]	validation_0-auc:0.790446	validation_1-auc:0.765046


XGBClassifier(colsample_bytree=0.9, gamma=0.1, learning_rate=0.08, max_depth=4,
              min_child_weight=150, n_estimators=120, n_jobs=-1, nthread=-1,
              reg_alpha=0.1, reg_lambda=0.1, seed=42, silent=True,
              subsample=0.9)

In [27]:
shap_values = xgb_model.get_booster().predict(xgb.DMatrix(dev_sample1[input_woe_variables]), pred_contribs=True)     
shap_df = pd.DataFrame(np.abs(shap_values[:,:-1]), columns=input_woe_variables)

shap_imp = shap_df.mean().sort_values(ascending=False).reset_index()
shap_imp.columns = ['Feature', 'Shap_Importance']
shap_imp = shap_imp[shap_imp['Shap_Importance']>0]

In [28]:
shap_imp.head(10)

,Feature,Shap_Importance
0,W_wb_v2_0_prob,0.258167
1,W_dxm_general_fraudc_v3_score,0.240363
2,W_limit_used_rate,0.225612
3,W_bw_b_score_v3,0.216126
4,W_tx_riskscore,0.190768
5,W_age,0.187881
6,W_ppde_7d_itfin_cashloan_cnt,0.102702
7,W_ppde_30d_itfin_cashloan_cnt,0.069378
8,W_diff_pass_credit_first_loan_time,0.067631
9,W_idcardpre6,0.058216


In [29]:
shap_imp.head(80)['Shap_Importance'].sum()/shap_imp['Shap_Importance'].sum()

0.9418915

In [30]:
sel_cols = [x for x in shap_imp.head(80)['Feature'].tolist()]

In [31]:
len(sel_cols)

80

In [32]:
import math
from six.moves import xrange
from catboost import Pool, CatBoostClassifier


class FocalLossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers with only __len__ and __getitem__ defined).
        # weights parameter can be None.
        # Returns list of pairs (der1, der2)
        gamma = 2.
        # alpha = 1.
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        exponents = []
        for index in xrange(len(approxes)):
            exponents.append(math.exp(approxes[index]))

        result = []
        for index in xrange(len(targets)):
            p = exponents[index] / (1 + exponents[index])

            if targets[index] > 0.0:
                der1 = -((1-p)**(gamma-1))*(gamma * math.log(p) * p + p - 1)/p
                der2 = gamma*((1-p)**gamma)*((gamma*p-1)*math.log(p)+2*(p-1))
            else:
                der1 = (p**(gamma-1)) * (gamma * math.log(1 - p) - p)/(1 - p)
                der2 = p**(gamma-2)*((p*(2*gamma*(p-1)-p))/(p-1)**2 + (gamma-1)*gamma*math.log(1 - p))

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))

        return result

In [112]:
from catboost import CatBoostClassifier

cat_params = {
    'iterations': 400, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'eval_metric':'AUC',
    'od_wait': 10, 
    'learning_rate': 0.05, 
    'reg_lambda': 2, 
    'random_strength': 4, 
    'random_state': 2023,
    'depth': 4, 
    'min_data_in_leaf': 415, 
    'leaf_estimation_iterations': 4, 
    'subsample': 0.9449700103203925,
    'verbose' : 1,
    'loss_function' : FocalLossObjective(),
}

cat_params =  {'bootstrap_type': 'Bernoulli', 'objective': 'CrossEntropy', 'eval_metric':'AUC', 'iterations': 400, 'od_wait': 6, 'random_strength': 4, 'max_bin': 4, 'reg_lambda': 7, 'subsample': 0.9218331876323019, 
               'learning_rate': 0.05, 'leaf_estimation_iterations': 3, 'depth': 3, 'random_state': 2023, 'min_data_in_leaf': 309, 'loss_function' : FocalLossObjective()
              }

cat_params =  {'bootstrap_type': 'Bernoulli', 'objective': 'CrossEntropy', 'eval_metric':'AUC', 'iterations': 400, 'loss_function' : FocalLossObjective(), 'od_wait': 7, 'random_strength': 4, 'max_bin': 5, 'reg_lambda': 8, 'subsample': 0.928854767323812, 'learning_rate': 0.05, 
 'leaf_estimation_iterations': 6, 'depth': 4, 'random_state': 2023, 'min_data_in_leaf': 485}

cat_params =  {'bootstrap_type': 'Bernoulli', 'objective': 'CrossEntropy', 'eval_metric':'AUC', 'iterations': 400, 'od_wait': 10, 'random_strength': 5, 
               'max_bin': 7, 'reg_lambda': 2, 'subsample': 0.8877390866014263, 'learning_rate': 0.05, 'leaf_estimation_iterations': 3, 'depth': 4, 'random_state': 2023, 'min_data_in_leaf': 467,
              'loss_function' : FocalLossObjective()
              }
# cat_params = {
#     'iterations': 400, 
#     'objective': 'CrossEntropy', 
#     'bootstrap_type': 'Bernoulli', 
#     'eval_metric':'AUC',
#     'od_wait': 4, 
#     'learning_rate': 0.05, 
#     'reg_lambda': 1, 
#     'random_strength': 4, 
#     'random_state': 42,
#     'depth': 4, 
#     'min_data_in_leaf': 234, 
#     'leaf_estimation_iterations': 7, 
#     'subsample': 0.8811664288639238,
#     'verbose' : 1,
#     'loss_function' : FocalLossObjective(),
# }

# {'bootstrap_type': 'Bernoulli', 'od_wait': 4, 'random_strength': 4, 'max_bin': 7, 'reg_lambda': 1, 'subsample': 0.8811664288639238, 
#  'learning_rate': 0.05, 'leaf_estimation_iterations': 7, 'depth': 4, 'random_state': 42, 'min_data_in_leaf': 234}

#  {'bootstrap_type': 'Bernoulli', 'od_wait': 6, 'random_strength': 6, 'max_bin': 7, 'reg_lambda': 1, 'subsample': 0.9004534526114775, 
#   'learning_rate': 0.05, 'leaf_estimation_iterations': 5, 'depth': 4, 'random_state': 42, 'min_data_in_leaf': 335}
    
# {'bootstrap_type': 'Bernoulli', 'od_wait': 10, 'random_strength': 4, 'max_bin': 4, 'reg_lambda': 2, 'subsample': 0.9449700103203925, 
#  'learning_rate': 0.05, 'leaf_estimation_iterations': 4, 'depth': 4, 'random_state': 2023, 'min_data_in_leaf': 415}

# cat_params = {'bootstrap_type': 'Bernoulli', 'eval_metric':'AUC', 'od_wait': 8, 'random_strength': 4, 'max_bin': 5, 'reg_lambda': 2, 'subsample': 0.9158264114777777, 'learning_rate': 0.05, 'leaf_estimation_iterations': 8, 'depth': 4, 'random_state': 42, 'min_data_in_leaf': 487}
# cat_params = {'bootstrap_type': 'Bernoulli','eval_metric':'AUC','od_wait': 10, 'random_strength': 5, 'max_bin': 5, 'reg_lambda': 2, 'subsample': 0.9816944236770894, 'learning_rate': 0.05, 'leaf_estimation_iterations': 4, 'depth': 4, 'random_state': 42, 'min_data_in_leaf': 434}
cat_model2 = CatBoostClassifier(**cat_params)

In [113]:
cat_model2.fit(train[sel_cols], train[target], eval_set=[(train[sel_cols], train[target]), (oot_sample1[sel_cols], oot_sample1[target])],
         early_stopping_rounds=20, verbose=20)

0:	test: 0.5575458	test1: 0.5552233	best: 0.5552233 (0)	total: 9.49ms	remaining: 3.79s
20:	test: 0.6943523	test1: 0.6614063	best: 0.6614063 (20)	total: 183ms	remaining: 3.3s
40:	test: 0.7352071	test1: 0.7017134	best: 0.7021537 (38)	total: 392ms	remaining: 3.43s
60:	test: 0.7499658	test1: 0.7173739	best: 0.7173739 (60)	total: 628ms	remaining: 3.49s
80:	test: 0.7539735	test1: 0.7196156	best: 0.7197563 (75)	total: 790ms	remaining: 3.11s
100:	test: 0.7601288	test1: 0.7254568	best: 0.7254568 (100)	total: 999ms	remaining: 2.96s
120:	test: 0.7634106	test1: 0.7282116	best: 0.7282116 (120)	total: 1.18s	remaining: 2.72s
140:	test: 0.7675793	test1: 0.7314276	best: 0.7314276 (140)	total: 1.35s	remaining: 2.48s
160:	test: 0.7703767	test1: 0.7337648	best: 0.7339035 (159)	total: 1.54s	remaining: 2.29s
180:	test: 0.7724666	test1: 0.7348176	best: 0.7348176 (180)	total: 1.77s	remaining: 2.14s
200:	test: 0.7748273	test1: 0.7363196	best: 0.7363196 (200)	total: 1.97s	remaining: 1.95s
220:	test: 0.7773517	t

In [114]:
cat_model2.get_params()

{'iterations': 400,
 'learning_rate': 0.05,
 'depth': 4,
 'loss_function': <__main__.FocalLossObjective at 0x7fc629132a10>,
 'od_wait': 10,
 'leaf_estimation_iterations': 3,
 'random_strength': 5,
 'eval_metric': 'AUC',
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.8877390866014263,
 'random_state': 2023,
 'reg_lambda': 2,
 'objective': 'CrossEntropy',
 'max_bin': 7,
 'min_data_in_leaf': 467}

In [115]:
model_data['ft3_2_proba_v2'] = cat_model2.predict_proba(model_data[sel_cols])[:,1]

In [116]:
import math

def score_card(prob):
    base_score = 600
    base_odds = 1/50
    PDO = 50
    B = PDO*1.0/math.log(2)
    A = base_score + B*math.log(base_odds)
    return round(A - B*math.log(prob/ (1-prob+1e-20)),0)

model_data['ft3_2_score_v2'] = model_data['ft3_2_proba_v2'].apply(score_card)

In [117]:
def source_map(x):
    if x<'2022-08':
        return '0.Train'
    else:
        return '1.OOT'

model_data['source'] = model_data['apply_month'].apply(lambda x: source_map(x))

In [119]:
def capture_topk(y_ture, y_pred, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    return np.sum(y_ture_sort[:topk])/np.sum(y_ture_sort)


def lift_topk(y_ture, y_pred, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    return (np.sum(y_ture_sort[:topk])/topk)/(np.sum(y_ture_sort)/len(y_ture))

# def lift_repair(data, cut_off, target, score, adjust_bad_rate, adjust_good_rate):
#     rj_sample_1 = data.query(f"{score} < @cut_off and {target}==1").shape[0]
#     rj_sample_0 = data.query(f"{score} < @cut_off and {target}==0").shape[0]
#     ps_sample_1 = data.query(f"{score} >= @cut_off and {target}==1").shape[0]
#     ps_sample_0 = data.query(f"{score} >= @cut_off and {target}==0").shape[0]
#     l = (rj_sample_1/adjust_bad_rate)/(rj_sample_1/adjust_bad_rate+rj_sample_0/adjust_good_rate)*\
#     (1+(rj_sample_0/adjust_good_rate +ps_sample_0/adjust_good_rate)/(rj_sample_1/adjust_bad_rate+ps_sample_1/adjust_bad_rate))
#     return l

def lift_repair_topk(y_ture, y_pred, adjust_bad_rate, adjust_good_rate, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    rj_sample_1 = np.sum(y_ture_sort[:topk])
    rj_sample_0 = topk - rj_sample_1
    ps_sample_1 = np.sum(y_ture_sort[topk:])
    ps_sample_0 = len(y_ture_sort[topk:]) - ps_sample_1
    return (rj_sample_1/adjust_bad_rate)/(rj_sample_1/adjust_bad_rate+rj_sample_0/adjust_good_rate)*\
    (1+(rj_sample_0/adjust_good_rate +ps_sample_0/adjust_good_rate)/(rj_sample_1/adjust_bad_rate+ps_sample_1/adjust_bad_rate))

def f_evalutor(x, proba_name, target):
    d = []
    d.append(x['cnt'].sum())
    d.append(x.query(f"{target}==[0, 1]")[target].sum())
    d.append(x.query(f"{target}==[0, 1]")[target].mean())
    d.append(x[target].replace(-1, 0).mean())
    d.append(round(roc_auc_score(x.query(f"{target}!=-1")[target], x.query(f"{target}!=-1")[proba_name]), 3))
    d.append(round(ks(x.query(f"{target}!=-1")[target], x.query(f"{target}!=-1")[proba_name]),3))
    d.append(round(roc_auc_score(x[target].replace(-1, 0), x[proba_name]),3))
    d.append(round(ks(x[target].replace(-1, 0), x[proba_name]),3))
    # capture@top10%
    d.append(round(capture_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 10), 3))
    # lift@top10%
    d.append(round(lift_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 10), 3))
    # lift_repair@top10%
    d.append(round(lift_repair_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 0.438, 0.16, 10), 3))
    return pd.Series(d, index=['#Count', '#Bad', '%Bad', '%Bad(灰当白)', 'AUC', 'KS', 'AUC(灰当白)', 'KS(灰当白)', 'Capture@top10%', 'Lift@top10%', 'Lift_Repair@top10%'])


model_data['cnt'] = 1
model_data.query("dpd30_at_mob6!=-9999").groupby(["source"]).apply(lambda x: f_evalutor(x, "ft3_2_proba_v2", "dpd30_at_mob6"))

,#Count,#Bad,%Bad,%Bad(灰当白),AUC,KS,AUC(灰当白),KS(灰当白),Capture@top10%,Lift@top10%,Lift_Repair@top10%
source,,,,,,,,,,,
0.Train,63365.0,5796.0,0.120441,0.091470,0.791,0.429,0.774,0.403,0.336,3.356,4.169
1.OOT,29918.0,2109.0,0.092642,0.070493,0.747,0.366,0.731,0.342,0.297,2.974,3.392


In [120]:
model_data['segment'] = model_data['loan_apply_count'].apply(lambda x: 1 if x==0 else 0)

In [121]:
model_data.query("dpd30_at_mob6!=-9999").groupby(["source", "segment"]).apply(lambda x: f_evalutor(x, "ft3_2_proba_v2", "dpd30_at_mob6"))

#Count    #Bad      %Bad  %Bad(灰当白)    AUC     KS  AUC(灰当白)  \
source  segment                                                                 
0.Train 0        11497.0  1262.0  0.150167   0.109768  0.802  0.447     0.774   
        1        51868.0  4534.0  0.114152   0.087414  0.787  0.425     0.774   
1.OOT   0         3880.0   355.0  0.123996   0.091495  0.749  0.377     0.728   
        1        26038.0  1754.0  0.088132   0.067363  0.744  0.360     0.729   

                 KS(灰当白)  Capture@top10%  Lift@top10%  Lift_Repair@top10%  
source  segment                                                            
0.Train 0          0.404           0.347        3.472               4.695  
        1          0.404           0.333        3.331               4.073  
1.OOT   0          0.351           0.279        2.792               3.296  
        1          0.337           0.293        2.931               3.309

In [122]:
model_data.query("dpd30_at_mob3!=-9999").groupby(["source", "segment"]).apply(lambda x: f_evalutor(x, "ft3_2_proba_v2", "dpd30_at_mob3"))

#Count    #Bad      %Bad  %Bad(灰当白)    AUC     KS  AUC(灰当白)  \
source  segment                                                                 
0.Train 0        11497.0   631.0  0.069608   0.054884  0.817  0.479     0.794   
        1        51868.0  1816.0  0.042586   0.035012  0.799  0.448     0.785   
1.OOT   0         4858.0   296.0  0.075820   0.060930  0.753  0.390     0.736   
        1        34259.0  1344.0  0.046782   0.039231  0.766  0.390     0.754   

                 KS(灰当白)  Capture@top10%  Lift@top10%  Lift_Repair@top10%  
source  segment                                                            
0.Train 0          0.441           0.439        4.392               5.209  
        1          0.426           0.410        4.097               4.483  
1.OOT   0          0.364           0.297        2.976               3.306  
        1          0.373           0.333        3.334               3.591

In [123]:
model_data.query("dpd15_at_mob1!=-9999").groupby(["source", "segment"]).apply(lambda x: f_evalutor(x, "ft3_2_proba_v2", "dpd15_at_mob1"))

#Count   #Bad      %Bad  %Bad(灰当白)    AUC     KS  AUC(灰当白)  \
source  segment                                                                
0.Train 0        11497.0  195.0  0.018370   0.016961  0.830  0.514     0.821   
        1        51868.0  391.0  0.008004   0.007538  0.794  0.446     0.788   
1.OOT   0         4858.0   78.0  0.017548   0.016056  0.779  0.433     0.771   
        1        34259.0  273.0  0.008458   0.007969  0.784  0.441     0.779   

                 KS(灰当白)  Capture@top10%  Lift@top10%  Lift_Repair@top10%  
source  segment                                                            
0.Train 0          0.500           0.554        5.541               5.855  
        1          0.437           0.422        4.220               4.291  
1.OOT   0          0.421           0.372        3.722               3.840  
        1          0.433           0.432        4.323               4.402

In [124]:
model_helper.model_group_monitor(model_data.query("dpd30_at_mob6!=[-9999, -1] and source=='1.OOT'"), "dpd30_at_mob6", "ft3_2_proba_v2", higher_better=False, number_of_groups=10)

,total_bad,total_good,total_bad_rate,cnt_group,cnt_bad,cnt_good,group_bad_rate,cum_bad,cum_good,cum_bad_pct,cum_good_pct,KS,ft3_2_proba_v2_min,ft3_2_proba_v2_max,ft3_2_proba_v2_avg
0,2109,20656,0.092642,2277,627,1650,0.275362,627,1650,0.297297,0.079880,0.217417,0.243189,0.857291,0.330424
1,2109,20656,0.092642,2276,376,1900,0.165202,1003,3550,0.475581,0.171863,0.303718,0.176695,0.243162,0.205553
2,2109,20656,0.092642,2277,316,1961,0.138779,1319,5511,0.625415,0.266799,0.358616,0.136296,0.176670,0.154943
3,2109,20656,0.092642,2276,218,2058,0.095782,1537,7569,0.728781,0.366431,0.362350,0.106878,0.136275,0.120698
4,2109,20656,0.092642,2276,179,2097,0.078647,1716,9666,0.813656,0.467951,0.345705,0.083013,0.106877,0.094612
5,2109,20656,0.092642,2277,152,2125,0.066755,1868,11791,0.885728,0.570827,0.314901,0.063972,0.082998,0.073353
6,2109,20656,0.092642,2276,106,2170,0.046573,1974,13961,0.935989,0.675881,0.260108,0.047371,0.063954,0.055488
7,2109,20656,0.092642,2277,66,2211,0.028986,2040,16172,0.967283,0.782920,0.184363,0.033003,0.047354,0.040256
8,2109,20656,0.092642,2276,52,2224,0.022847,2092,18396,0.991939,0.890589,0.101351,0.019656,0.033002,0.026150
9,2109,20656,0.092642,2277,17,2260,0.007466,2109,20656,1.000000,1.000000,0.000000,0.000720,0.019648,0.011907


In [125]:
pickle.dump(cat_model2, open("../input/ft3_2_cat_model_2.pkl", "wb"))

In [131]:
import joblib
import math

def score_card(prob):
    base_score = 600
    base_odds = 1/50
    PDO = 50
    B = PDO*1.0/math.log(2)
    A = base_score + B*math.log(base_odds)
    return round(A - B*math.log( prob/ (1-prob+1e-20)),0)

def lgb_pkl_score(df, model_pkl_path,score_name,predict_pos_neg=1,return_list=[],both_score_with_prob=True):
    model = joblib.load(open(model_pkl_path,"rb"))
    model_features_list = model.feature_name_
    df[model_features_list] = df[model_features_list].replace(['none', 'none\t', -9999, -99, -999, -88888, -99999, -9999999, -2, -1], np.nan)
    model_result = df[return_list]
    model_result[score_name] = model.predict_proba(df[model_features_list])[:, predict_pos_neg]
    if both_score_with_prob:
        model_result[score_name.replace("prob","score")] = model_result[score_name].apply(score_card)
    return model_result

In [132]:
model_data[['ft_v3_2_quick_prob', 'ft_v3_2_quick_score']] = lgb_pkl_score(df=model_data, model_pkl_path='../model/ft_v3_2_quick.mdl.pkl',score_name='ft_v3_2_quick_prob',predict_pos_neg=1,both_score_with_prob=True).values

In [133]:
def capture_topk(y_ture, y_pred, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    return np.sum(y_ture_sort[:topk])/np.sum(y_ture_sort)


def lift_topk(y_ture, y_pred, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    return (np.sum(y_ture_sort[:topk])/topk)/(np.sum(y_ture_sort)/len(y_ture))

# def lift_repair(data, cut_off, target, score, adjust_bad_rate, adjust_good_rate):
#     rj_sample_1 = data.query(f"{score} < @cut_off and {target}==1").shape[0]
#     rj_sample_0 = data.query(f"{score} < @cut_off and {target}==0").shape[0]
#     ps_sample_1 = data.query(f"{score} >= @cut_off and {target}==1").shape[0]
#     ps_sample_0 = data.query(f"{score} >= @cut_off and {target}==0").shape[0]
#     l = (rj_sample_1/adjust_bad_rate)/(rj_sample_1/adjust_bad_rate+rj_sample_0/adjust_good_rate)*\
#     (1+(rj_sample_0/adjust_good_rate +ps_sample_0/adjust_good_rate)/(rj_sample_1/adjust_bad_rate+ps_sample_1/adjust_bad_rate))
#     return l

def lift_repair_topk(y_ture, y_pred, adjust_bad_rate, adjust_good_rate, top=20):
    sort_index = np.argsort(-y_pred)
    y_ture_sort = y_ture[sort_index]
    topk = int(len(y_pred)*top/100)
    rj_sample_1 = np.sum(y_ture_sort[:topk])
    rj_sample_0 = topk - rj_sample_1
    ps_sample_1 = np.sum(y_ture_sort[topk:])
    ps_sample_0 = len(y_ture_sort[topk:]) - ps_sample_1
    return (rj_sample_1/adjust_bad_rate)/(rj_sample_1/adjust_bad_rate+rj_sample_0/adjust_good_rate)*\
    (1+(rj_sample_0/adjust_good_rate +ps_sample_0/adjust_good_rate)/(rj_sample_1/adjust_bad_rate+ps_sample_1/adjust_bad_rate))

def f_evalutor(x, proba_name, target):
    d = []
    d.append(x['cnt'].sum())
    d.append(x.query(f"{target}==[0, 1]")[target].sum())
    d.append(x.query(f"{target}==[0, 1]")[target].mean())
    d.append(x[target].replace(-1, 0).mean())
    d.append(round(roc_auc_score(x.query(f"{target}!=-1")[target], x.query(f"{target}!=-1")[proba_name]), 3))
    d.append(round(ks(x.query(f"{target}!=-1")[target], x.query(f"{target}!=-1")[proba_name]),3))
    d.append(round(roc_auc_score(x[target].replace(-1, 0), x[proba_name]),3))
    d.append(round(ks(x[target].replace(-1, 0), x[proba_name]),3))
    # capture@top10%
    d.append(round(capture_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 10), 3))
    # lift@top10%
    d.append(round(lift_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 10), 3))
    # lift_repair@top10%
    d.append(round(lift_repair_topk(x.query(f"{target}!=-1")[target].values, x.query(f"{target}!=-1")[proba_name].values, 0.438, 0.16, 10), 3))
    return pd.Series(d, index=['#Count', '#Bad', '%Bad', '%Bad(灰当白)', 'AUC', 'KS', 'AUC(灰当白)', 'KS(灰当白)', 'Capture@top10%', 'Lift@top10%', 'Lift_Repair@top10%'])


model_data['cnt'] = 1
model_data.query("dpd30_at_mob6!=-9999").groupby(["source"]).apply(lambda x: f_evalutor(x, "ft_v3_2_quick_prob", "dpd30_at_mob6"))

,#Count,#Bad,%Bad,%Bad(灰当白),AUC,KS,AUC(灰当白),KS(灰当白),Capture@top10%,Lift@top10%,Lift_Repair@top10%
source,,,,,,,,,,,
0.Train,63365.0,5796.0,0.120441,0.091470,0.772,0.403,0.753,0.376,0.309,3.09,3.737
1.OOT,29918.0,2109.0,0.092642,0.070493,0.728,0.330,0.710,0.304,0.268,2.68,2.994


In [126]:
input_cols = [x[2:] for x in cat_model2.feature_names_]

In [127]:
result = """"""
result += "def process(model_data):\n"
result += "  import numpy as np\n"
for col in input_cols:
    bins_info_temp = mapiv_temp.query(f"varname=='{col}'")
    have_null = 1-int(bins_info_temp['bin'].tolist()[0])
    # woe transformer
    for index, left, right, woe in zip(bins_info_temp['bin'].tolist(), 
                                bins_info_temp['ll'].tolist(),
                                bins_info_temp['ul'].tolist(),
                                bins_info_temp['woe'].tolist()):
        
        if have_null:
            if str(left) == "nan" and str(right) == "nan":
                result += f"  model_data['W_{col}'] = \\\n    np.where(model_data['{col}'].isnull(),     {woe},\n"
            elif str(right) == "inf":
                result += f"  {woe}" + ")"*int(index) + "\n\n"
            else:
                result += f"    np.where(model_data['{col}'] < {right},        {woe},\n"
        else:
            if int(index)==1:
                result += f"  model_data['W_{col}'] = \\\n    np.where(model_data['{col}'] <{right},     {woe},\n"
            elif str(right) == "inf":
                result += f"  {woe}" + ")"*(int(index)-1) + "\n\n"
            else:
                result += f"    np.where(model_data['{col}'] < {right},        {woe},\n"
            
result += "  return model_data"
 
print(result, file=open("condition_calculate_mob6_input_v2.py", "w+"))

In [128]:
sel_cols

['W_wb_v2_0_prob',
 'W_dxm_general_fraudc_v3_score',
 'W_limit_used_rate',
 'W_bw_b_score_v3',
 'W_tx_riskscore',
 'W_age',
 'W_ppde_7d_itfin_cashloan_cnt',
 'W_ppde_30d_itfin_cashloan_cnt',
 'W_diff_pass_credit_first_loan_time',
 'W_idcardpre6',
 'W_jd_ppre_debit_level',
 'W_als_m3_id_nbank_nsloan_orgnum',
 'W_als_m12_id_nbank_cons_allnum',
 'W_als_m1_cell_nbank_selfnum',
 'W_als_m3_id_rel_allnum',
 'W_loan_term6_apply_count',
 'W_youmeng_bank_score',
 'W_als_m6_id_nbank_nsloan_orgnum',
 'W_jd_ppre_credit_asst_level',
 'W_als_m1_id_nbank_selfnum',
 'W_als_m1_id_nbank_nsloan_allnum',
 'W_als_m3_id_min_inteday',
 'W_als_m6_id_min_monnum',
 'W_als_m12_cell_nbank_cons_allnum',
 'W_als_m6_id_bank_tot_mons',
 'W_als_m3_id_nbank_min_inteday',
 'W_ronghui_p_ft_p0136',
 'W_gender',
 'W_ppde_60d_itfin_cashloan_cnt',
 'W_ppde_3d_itfin_cashloan_cnt',
 'W_als_m3_id_nbank_sloan_allnum',
 'W_als_m3_cell_min_inteday',
 'W_als_m12_id_nbank_week_orgnum',
 'W_als_m6_id_nbank_cons_allnum',
 'W_als_m6_id_

In [129]:
model_data[['loan_id', 'ft3_2_proba_v2']].to_csv("../output/ft3_2_model_data_ft3_2_score_v2.csv", index=False)

In [110]:
def objective(trial, train=train, valid=validation, oot=oot_sample1, sel_cols = sel_cols, target=target):
    cat_params = {
        'eval_metric':'AUC',
        'objective': 'CrossEntropy', 
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli']),
        'loss_function' : FocalLossObjective(),
        'od_wait': trial.suggest_int('od_wait', 3, 10),
        'random_strength':trial.suggest_categorical('random_strength', [4, 5, 6]),
        'max_bin': trial.suggest_int('max_bin', 2, 8),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.88, 0.99),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.05]),
        'leaf_estimation_iterations':trial.suggest_int('leaf_estimation_iterations', 3, 10),
        'iterations':  400,
        'depth': trial.suggest_categorical('depth', [4]),
        'random_seed': trial.suggest_categorical('random_state', [42, 2023]),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 200, 500),
    }
    cat_model = CatBoostClassifier(**cat_params)
    
    cat_model.fit(train[sel_cols], train[target], eval_set=[(train[sel_cols], train[target]), (oot_sample1[sel_cols], oot_sample1[target])],
             early_stopping_rounds=20, verbose=False)   
    oot['proba'] = cat_model.predict_proba(oot_sample1[sel_cols])[:,1]
    
    auc_score = lift_topk(oot[target].values, oot["proba"].values, 10)
    return auc_score

In [111]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-05-11 12:50:12,938] A new study created in memory with name: no-name-d93631e0-7072-4901-b279-721b7d93956e
[I 2023-05-11 12:50:16,214] Trial 0 finished with value: 2.869288329120907 and parameters: {'bootstrap_type': 'Bernoulli', 'od_wait': 5, 'random_strength': 6, 'max_bin': 5, 'reg_lambda': 9, 'subsample': 0.9340242322921329, 'learning_rate': 0.05, 'leaf_estimation_iterations': 8, 'depth': 4, 'random_state': 2023, 'min_data_in_leaf': 310}. Best is trial 0 with value: 2.869288329120907.
[I 2023-05-11 12:50:22,042] Trial 1 finished with value: 2.935685083844366 and parameters: {'bootstrap_type': 'Bernoulli', 'od_wait': 8, 'random_strength': 4, 'max_bin': 6, 'reg_lambda': 10, 'subsample': 0.956889286054307, 'learning_rate': 0.05, 'leaf_estimation_iterations': 8, 'depth': 4, 'random_state': 2023, 'min_data_in_leaf': 440}. Best is trial 1 with value: 2.935685083844366.
[I 2023-05-11 12:50:24,889] Trial 2 finished with value: 2.883516205133077 and parameters: {'bootstrap_type': 'Ber

KeyboardInterrupt: 